# Exhaustive Optimizer Log Example

When evaluating optimization performance it can be useful to observe the parametric surface of optimization results. The `ITKOptimizationMonitor` external module provides an event-based procedure through which data generated with an `itk.ExhaustiveOptimizer` may be preserved and referenced as an `itk.Image` object.

This example adapts the [ITKSphinxExamples Exhaustive Optimizer sample notebook](https://itk.org/ITKExamples/src/Numerics/Optimizers/ExhaustiveOptimizer/PlotExhaustiveOptimizer.html) to demonstrate how the `itk.CommandExhaustiveLog` class may be employed for data collection and surface visualization. Two 2D images are registered with an `itk.Euler2DTransform` evaluated with an `itk.MeanSquaresImageToImageMetricv4` object. Results are visualized over the entire optimization region as a 3D image and then as a 2D image slice.

In [1]:
import os
from urllib.request import urlretrieve

import itk
from itkwidgets import view

### Get sample data

In [2]:
os.makedirs('Input', exist_ok=True)

In [3]:
fixed_img_path = 'Input/apple.jpg'
moving_img_path = 'Input/orange.jpg'

In [4]:
if not os.path.exists(fixed_img_path):
    url = 'https://data.kitware.com/api/v1/file/5cad1aec8d777f072b181870/download'
    urlretrieve(url, fixed_img_path)
if not os.path.exists(moving_img_path):
    url = 'https://data.kitware.com/api/v1/file/5cad1aed8d777f072b181879/download'
    urlretrieve(url, moving_img_path)

In [5]:
fixed_img = itk.imread(fixed_img_path, itk.F)
moving_img = itk.imread(moving_img_path, itk.F)

### Define Registration Parameters

In [6]:
dimension = 2
FixedImageType = itk.Image[itk.F, dimension]
MovingImageType = itk.Image[itk.F, dimension]
TransformType = itk.Euler2DTransform[itk.D]
OptimizerType = itk.ExhaustiveOptimizerv4[itk.D]
MetricType = itk.MeanSquaresImageToImageMetricv4[FixedImageType, MovingImageType]
TransformInitializerType = \
    itk.CenteredTransformInitializer[itk.MatrixOffsetTransformBase[itk.D,2,2],
                                     FixedImageType, MovingImageType]
RegistrationType = itk.ImageRegistrationMethodv4[FixedImageType,MovingImageType]

In [7]:
transform = TransformType.New()

initializer = TransformInitializerType.New(
    Transform=transform,
    FixedImage=fixed_img,
    MovingImage=moving_img,
)
initializer.InitializeTransform()

In [8]:
metric_results = dict()

metric = MetricType.New()
optimizer = OptimizerType.New()

optimizer.SetNumberOfSteps([10,10,1])

scales = optimizer.GetScales()
scales.SetSize(3)
scales.SetElement(0, 0.1)
scales.SetElement(1, 1.0)
scales.SetElement(2, 1.0)
optimizer.SetScales(scales)

registration = RegistrationType.New(Metric=metric,
    Optimizer=optimizer,
    FixedImage=fixed_img,
    MovingImage=moving_img,
    InitialTransform=transform,
    NumberOfLevels=1)

### Add CommandExhaustiveLog Observer

Note that available `itk.CommandExhaustiveLog` template wrappings are limited to wrapped ITK image types. In order to visualize optimization over a transform with greater than three parameters it may be necessary to build ITK locally from source with extra image wrappings.

In [9]:
observer = itk.CommandExhaustiveLog[itk.F, transform.GetNumberOfParameters()].New()
observer.SetCenter(transform.GetParameters())

optimizer.AddObserver(itk.StartEvent(), observer)      # Initialize
optimizer.AddObserver(itk.IterationEvent(), observer)  # Collect data

1

### Run Registration

Observer data is updated with each `itk.IterationEvent` fired by the optimizer.

In [10]:
registration.Update()

print(f'MinimumMetricValue: {optimizer.GetMinimumMetricValue():.4f}\t'
      f'MaximumMetricValue: {optimizer.GetMaximumMetricValue():.4f}\n'
      f'MinimumMetricValuePosition: {list(optimizer.GetMinimumMetricValuePosition())}\t'
      f'MaximumMetricValuePosition: {list(optimizer.GetMaximumMetricValuePosition())}\n'
      f'StopConditionDescription: {optimizer.GetStopConditionDescription()}\t')

MinimumMetricValue: 8174.5320	MaximumMetricValue: 13966.5686
MinimumMetricValuePosition: [0.0, 0.0, 1.0]	MaximumMetricValuePosition: [0.6000000000000001, 10.0, -1.0]
StopConditionDescription: ExhaustiveOptimizerv4: Completed sampling of parametric space of size 3	


### Visualize 3D Parameter Space

Metric values within the exhaustive transform region may be visualized directly with `itkwidgets` as a 3D `itk.Image` object.

In [11]:
view(observer.GetDataImage())

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

### Visualize 2D Data Slice 
Metric values may be visualized over a 2D slice of the 3D transform parameter domain using `itkwidgets`. In this case we will slice along the third transform dimension to visualize the surface containing the maximum metric value in the exhaustive domain.

See [Process a 2D Slice of a 3D Image](https://itk.org/ITKExamples/src/Filtering/ImageGrid/ProcessA2DSliceOfA3DImage/Documentation.html) in the [ITKSphinxExamples](https://itk.org/ITKExamples/index.html) for image slicing procedures.

In [12]:
max_point = optimizer.GetMaximumMetricValuePosition()

In [13]:
filter = itk.ExtractImageFilter[type(observer.GetDataImage()),itk.Image[itk.F,2]].New()
filter.SetInput(observer.GetDataImage())
filter.SetDirectionCollapseToSubmatrix()

In [14]:
dest_region = observer.GetDataImage().GetBufferedRegion()
size = dest_region.GetSize()
index = dest_region.GetIndex()

size[2] = 0    # Collapse along third dimension
index[2] = observer.GetDataImage().TransformPhysicalPointToIndex(max_point)[2]

dest_region.SetSize(size)
dest_region.SetIndex(index)
filter.SetExtractionRegion(dest_region)

In [15]:
filter.Update()

In [16]:
view(filter.GetOutput())

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF2; pro…